notebook is sm_ont-process_reads


envs used = nanofilt, n50, minimap2, samtools, mosdepth, samtools, mosdepth, cutesv

In [1]:
import os
from pathlib import Path
import pandas as pd

In [2]:
proj_dir = "/master/nplatt/sm_nanopore"
ref_fas = "{}/data/genome/GCA_000237925.5.fa".format(proj_dir)
Path("{}/results".format(proj_dir)).mkdir(parents=True, exist_ok=True)

In [3]:
samples = [ "smor", 
            "smle_pzq_es",
            "smle_pzq_er",
            "smbre", 
            "smeg"        ]

In [4]:
os.chdir("{}/results".format(proj_dir))

## Filter and map raw reads

In [ ]:
#all raw reads are provided in results/raw_data

In [6]:
Path("{}/results/nanofilt".format(proj_dir)).mkdir(parents=True, exist_ok=True)
os.chdir("{}/results/nanofilt".format(proj_dir))

In [43]:
for sample in samples:
    raw_fq = "{}/results/raw_reads/{}.raw_pass.fastq".format(proj_dir, sample)

    !~/anaconda3/envs/sm_ont-process_reads/bin/NanoFilt --length 500 --quality 7 --headcrop 25 --tailcrop 25 {raw_fq} >{sample}.filtered.fastq

In [ ]:
!conda run -n n50 --cwd . --live-stream n50 --format csv $(ls *.filtered.fastq) >n50_filtered_reads.csv

In [46]:
n50_df = pd.read_csv("n50_filtered_reads.csv", sep=",")
n50_df

,#path,seqs,size,N50,min,max,N75,N90,auN
0,smbre.filtered.fastq,2156891,13733527914,16145,500,193069,7657,2728,357349.74
1,smeg.filtered.fastq,2096508,12631573135,12491,500,236942,5659,2587,399825.88
2,smle_pzq_er.filtered.fastq,1330051,7000372416,12071,500,447354,4953,2105,227185.13
3,smor.filtered.fastq,3413369,14119478724,8668,500,168273,3430,1618,598308.11
4,smle_pzq_es.filtered.fastq,3662716,23840253863,13824,500,202196,5893,2659,624239.05


In [7]:
Path("{}/results/minimap2".format(proj_dir)).mkdir(parents=True, exist_ok=True)
os.chdir("{}/results/minimap2".format(proj_dir))

In [49]:
# for sample in samples:
for sample in ["smle_pzq_es"]:
    sam      = "{}/results/minimap2/{}.sam".format(proj_dir, sample)
    query_fq = "{}/results/nanofilt/{}.filtered.fastq".format(proj_dir, sample)

    print("###################### {} ##################".format(sample))
    !~/anaconda3/envs/sm_ont-process_reads/bin/minimap2 -a -o {sam} -t 48 -x map-ont {ref_fas} {query_fq}

###################### smle_pzq_es ##################
[M::mm_idx_gen::11.489*1.68] collected minimizers
[M::mm_idx_gen::13.279*2.42] sorted minimizers
[M::main::13.279*2.42] loaded/built the index for 12 target sequence(s)
[M::mm_mapopt_update::14.118*2.34] mid_occ = 416
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 12
[M::mm_idx_stat::14.681*2.28] distinct minimizers: 30119087 (65.35% are singletons); average occurrences: 2.439; average spacing: 5.327; total length: 391422238
[M::worker_pipeline::214.298*29.98] mapped 42338 sequences
[M::worker_pipeline::390.138*31.90] mapped 40808 sequences
[M::worker_pipeline::588.947*32.25] mapped 43306 sequences
[M::worker_pipeline::830.022*32.30] mapped 45455 sequences
[M::worker_pipeline::1051.682*32.12] mapped 44703 sequences
[M::worker_pipeline::1289.362*31.59] mapped 45693 sequences
[M::worker_pipeline::1470.012*32.22] mapped 47806 sequences
[M::worker_pipeline::1696.452*32.12] mapped 45932 sequences
[M::worker_pipeline::1916.264

In [50]:
# for sample in samples:
for sample in ["smle_pzq_es"]:
    sam      = "{}/results/minimap2/{}.sam".format(proj_dir, sample)
    query_fq = "{}/results/nanofilt/{}.filtered.fastq".format(proj_dir, sample)
    
    print("###################### {} ##################".format(sample))

    #sort bam
    print("...sort...")
    ! ~/anaconda3/envs/sm_ont-process_reads/bin/samtools sort -O bam -o {sample}.bam {sample}.sam
    
    #index bam
    print("...index...")
    ! ~/anaconda3/envs/sm_ont-process_reads/bin/samtools index {sample}.bam
    
    #calculate coverage
    print("...coverage...")
    ! ~/anaconda3/envs/sm_ont-process_reads/bin/mosdepth --threads 4 {sample} {sample}.bam


###################### smle_pzq_es ##################
...sort...
[bam_sort_core] merging from 60 files and 1 in-memory blocks...
...index...
...coverage...


In [8]:
#calculate coverage for a table
!echo "sample,length,bases,mean,min,max">covs.csv
!grep "total" *mosdepth.summary.txt | sed 's/:/\t/' | sed 's/\t/,/g' | cut -f1,3-7 -d"," | sed 's/.mosdepth.summary.txt//' >>covs.csv

cov_df=pd.read_csv("covs.csv", sep=",")
cov_df

,sample,length,bases,mean,min,max
0,smbre,391422238,12929849971,33.03,0,50586
1,smeg,391422238,8667326476,22.14,0,34407
2,smle_pzq_er,391422238,6624879196,16.93,0,10350
3,smle_pzq_es,391422238,21017040798,53.69,0,43602
4,smor,391422238,12178154968,31.11,0,80622


In [ ]:
#play around with igv

## Identify structural variants

In [9]:
Path("{}/results/cutesv".format(proj_dir)).mkdir(parents=True, exist_ok=True)
os.chdir("{}/results/cutesv".format(proj_dir))

In [5]:
#sv call
# for sample in samples:
for sample in ["smle_pzq_es"]:

    print("###################### {} ##################".format(sample))

    bam      = "{}/results/minimap2/{}.bam".format(proj_dir, sample)
    query_fq = "{}/results/nanofilt/{}.filtered.fastq.gz".format(proj_dir, sample)
    out_dir  = "{}/{}".format(os.getcwd(), sample)
    
    Path("{}".format(out_dir)).mkdir(parents=True, exist_ok=True)
    !~/anaconda3/envs/cutesv/bin/cuteSV --max_cluster_bias_INS 100 --diff_ratio_merging_INS 0.3 --max_cluster_bias_DEL 100 --diff_ratio_merging_DEL 0.3 --threads 12 --min_support 10 --genotype {bam} {ref_fas} {sample}.cutesv.vcf {out_dir}


###################### smle_pzq_es ##################
2023-01-18 11:31:03,178 [INFO] Running /master/nplatt/anaconda3/envs/cutesv/bin/cuteSV --max_cluster_bias_INS 100 --diff_ratio_merging_INS 0.3 --max_cluster_bias_DEL 100 --diff_ratio_merging_DEL 0.3 --threads 12 --min_support 10 --genotype /master/nplatt/sm_nanopore/results/minimap2/smle_pzq_es.bam /master/nplatt/sm_nanopore/data/genome/GCA_000237925.5.fa smle_pzq_es.cutesv.vcf /master/nplatt/sm_nanopore/results/smle_pzq_es
2023-01-18 11:31:10,070 [INFO] The total number of chromsomes: 12
2023-01-18 11:31:38,245 [INFO] Finished OU426849.1:10000000-10680109.
2023-01-18 11:33:46,325 [INFO] Finished HE601624.3:80000000-87984036.
2023-01-18 11:34:13,522 [INFO] Finished HE601624.3:30000000-40000000.
2023-01-18 11:34:14,461 [INFO] Finished HE601624.3:60000000-70000000.
2023-01-18 11:34:14,861 [INFO] Finished HE601624.3:50000000-60000000.
2023-01-18 11:34:15,292 [INFO] Finished HE601624.3:40000000-50000000.
2023-01-18 11:34:15,396 [INFO] F

## Read Correction

In [ ]:
Path("{}/results/read_correction".format(proj_dir)).mkdir(parents=True, exist_ok=True)
os.chdir("{}/results/read_correction".format(proj_dir))

for sample in samples:
    print("###################### {} ##################".format(sample))
    Path("{}/results/read_correction/{}".format(proj_dir, sample)).mkdir(parents=True, exist_ok=True)
    
    os.chdir("{}/results/read_correction".format(proj_dir, sample))
    
    !conda run -n ont_assembly --cwd . --live-stream canu -correct -genomesize=385m -useGrid=False -p {sample}  -nanopore {proj_dir}/results/raw_reads/{sample}.raw_pass.fastq
    
    os.chdir("{}/results/read_correction".format(proj_dir))

In [ ]:
# %%bash

for SAMPLE in 'smor' 'smle_pzq_es' 'smle_pzq_er' 'smbre' 'smeg'; do

    if [ -d ${SAMPLE} ]; then
        rm -rf ${SAMPLE}
    fi
    
    mkdir ${SAMPLE}
    cd ${SAMPLE}
    
    LOG=$(pwd)/${SAMPLE}.log 
    
    QSUB="qsub -V -cwd -S /bin/bash -q all.q -j y -pe smp 192 -N ${SAMPLE} -o ${LOG}"
    CMD="conda run -n ont_assembly --cwd . --live-stream canu -correct -genomesize=385m -useGrid=False -p ${SAMPLE}  -nanopore /master/nplatt/sm_nanopore/results/raw_reads/${SAMPLE}.raw_pass.fastq"
    
    echo $CMD | $QSUB
    cd ..
done

for SAMPLE in 'smor' 'smle_pzq_es' 'smbre'; do
    rm -rf $SAMPLE*
    QSUB="qsub -V -cwd -S /bin/bash -q all.q -j y -pe smp 192 -N ${SAMPLE} -o ${SAMPLE}.log"
    CMD="conda run -n ont_assembly --cwd . --live-stream canu -correct -genomesize=385m -useGrid=False -p ${SAMPLE}/${SAMPLE}  -nanopore /master/nplatt/sm_nanopore/results/raw_reads/${SAMPLE}.raw_pass.fastq"
    
    echo $CMD | $QSUB
done

## Genome assembly

In [41]:
Path("{}/results/assembly".format(proj_dir)).mkdir(parents=True, exist_ok=True)
os.chdir("{}/results/assembly".format(proj_dir))

In [ ]:
for sample in samples:
    print("###################### {} ##################".format(sample))
    !flye --nano-cor {proj_dir}/results/read_correction/{sample}.correctedReads.fasta.gz -g 385m -t 128 -i 3 -o {sample}_rnd0

In [ ]:
for SAMPLE in 'smor' 'smle_pzq_es' 'smle_pzq_er' 'smbre' 'smeg'; do

    if [ -d ${SAMPLE} ]; then
        rm -rf ${SAMPLE}
    fi
    
    mkdir ${SAMPLE}
    cd ${SAMPLE}
    
    LOG=$(pwd)/${SAMPLE}.log 
    READS="/master/nplatt/sm_nanopore/results/read_correction/${SAMPLE}/${SAMPLE}.correctedReads.fasta.gz"
    OUT="/master/nplatt/sm_nanopore/results/assembly/${SAMPLE}/${SAMPLE}_rnd0"
    
    QSUB="qsub -V -cwd -S /bin/bash -q all.q -j y -pe smp 128 -N ${SAMPLE} -o ${LOG}"
    CMD="conda run -n sm_ont-process_reads --cwd . --live-stream flye --nano-cor ${READS} -g 385m -t 128 -i 3 -o ${OUT}"
    
    echo $CMD | $QSUB
    cd ..
done

In [37]:
#get df of assembly stats
total_lengths=[]
fragments=[]
fragment_n50s=[]
largest_frags=[]
scaffolds=[]
mean_coverages=[]
    
for sample in samples:
    info = !tail -n 8 {sample}/{sample}.log | head -n 6
    total_lengths.append(info[0].split("\t")[-1])
    fragments.append(info[1].split("\t")[-1])
    fragment_n50s.append(info[2].split("\t")[-1])
    largest_frags.append(info[3].split("\t")[-1])
    scaffolds.append(info[4].split("\t")[-1])
    mean_coverages.append(info[5].split("\t")[-1])
    
assembly_df=pd.DataFrame(data = [samples, total_lengths, fragments, fragment_n50s, largest_frags, scaffolds, mean_coverages]).T
assembly_df.columns=["sample", "total_length", "fragments", "fragment_n50", "largest_fragment", "scaffolds", "mean_coverage"]
assembly_df.to_csv("assembly_info.csv", index=False)
assembly_df

,sample,total_length,fragments,fragment_n50,largest_fragment,scaffolds,mean_coverage
0,smor,386656054,1674,2150179,8090473,0,23
1,smle_pzq_es,403425661,1122,3481572,14813648,0,33
2,smle_pzq_er,382213358,1968,1273691,5768858,0,15
3,smbre,384951218,1020,2298594,14522024,0,24
4,smeg,370379461,1963,871323,6130221,0,14


In [45]:
for sample in samples:
    print(sample)
    ! cp /master/nplatt/sm_nanopore/results/assembly/{sample}/{sample}_rnd0/assembly.fasta {sample}_rnd0.fasta

smor
smle_pzq_es
smle_pzq_er
smbre
smeg


## polish assembly

In [46]:
Path("{}/results/polish_assembly".format(proj_dir)).mkdir(parents=True, exist_ok=True)
os.chdir("{}/results/polish_assembly".format(proj_dir))

In [80]:
for sample in samples:
    print(sample)
    !ln -s /master/nplatt/sm_nanopore/results/assembly/{sample}_rnd0.fasta {sample}.assembly_rnd0.fasta

smor
smle_pzq_es
smle_pzq_er
smbre
smeg


In [81]:
#set the initial round
rnd=1
for sample in samples:
    print("###################### {} ##################".format(sample))

    for rnd in [1, 2, 3]:
        #define major i/o files
        current_assembly = "{}.assembly_rnd{}.fasta".format(sample, str(rnd-1))
        corrected_reads = "{}/results/read_correction/{}/{}.correctedReads.fasta.gz".format(proj_dir, sample, sample)
        polished_assembly = "{}.assembly_rnd{}.fasta".format(sample, str(rnd))
        n_threads=48
        
        #map corrected reads to the assembly
        print("R{}: map".format(rnd))
        minimap_cmd = "conda run -n sm_ont-process_reads --cwd . --live-stream \
            minimap2 \
                -a \
                -map-ont \
                -t {} \
                {} \
                {}  \
                >{}.racon_rnd{}.sam".format(n_threads, current_assembly, corrected_reads, sample, rnd)
        minimap_cmd=' '.join(minimap_cmd.split())
        ! {minimap_cmd}

        #polish the assembly
        print("R{}: polish".format(rnd))
        racon_cmd = "conda run -n sm_ont-process_reads --cwd . --live-stream \
            racon \
                -t {} \
                {} \
                {}.racon_rnd{}.sam \
                {} \
                >{}".format(n_threads, corrected_reads, sample, rnd, current_assembly, polished_assembly)
        racon_cmd=' '.join(racon_cmd.split())
        ! {racon_cmd}
        print("#")

###################### smor ##################
R1: map
[M::mm_idx_gen::7.984*1.68] collected minimizers
[M::mm_idx_gen::9.287*2.93] sorted minimizers
[M::main::9.288*2.93] loaded/built the index for 1674 target sequence(s)
[M::mm_mapopt_update::10.191*2.76] mid_occ = 384
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 1674
[M::mm_idx_stat::10.857*2.65] distinct minimizers: 30374815 (65.19% are singletons); average occurrences: 2.391; average spacing: 5.323; total length: 386656054
[M::worker_pipeline::152.226*39.59] mapped 79894 sequences
[M::worker_pipeline::307.705*40.33] mapped 77039 sequences
[M::worker_pipeline::476.213*40.86] mapped 74651 sequences
[M::worker_pipeline::629.258*41.74] mapped 76580 sequences
[M::worker_pipeline::794.384*41.56] mapped 73726 sequences
[M::worker_pipeline::965.772*41.19] mapped 76750 sequences
[M::worker_pipeline::1144.195*40.83] mapped 76077 sequences
[M::worker_pipeline::1289.656*41.18] mapped 77343 sequences
[M::worker_pipeline::1453.221

In [ ]:
## Genome contamination 

In [ ]:
mkdir -p ${PROJ_DIR}/data/remove_contamination && cd ${PROJ_DIR}/data/remove_contamination

In [ ]:
#https://github.com/timkahlke/BASTA/wiki/3.-BASTA-Usage

In [ ]:
%%bash 

conda activate basta_py3

mkdir -p ${PROJ_DIR}/data/remove_contamination && cd ${PROJ_DIR}/data/remove_contamination

genome_updater.sh \
    -g "" \
    -d "refseq" \
    -c "representative genome" \
    -l "complete genome,chromosome" \
    -f "genomic.fna.gz,assembly_report.txt" \
    -o "refseq_216_fas" \
    -b "v1" \
    -a \
    -A species:1 \
    -m \
    -u \
    -r \
    -p \
    -t 48

#uncompress for kraken2
cd refseq_216_fas/v1/files
    
ls *.gz | parallel -j 48 gunzip {}

In [ ]:
#create blast database
cat *.fas >refseq_216.fas

In [ ]:
makeblastdb -dbtype nucle -in 

In [ ]:
blastm -outfmt 6 -in <assembly> -out <assembly>.blastn --threads 48

In [ ]:
./bin/basta taxonomy

In [ ]:
./bin/basta download gb

In [ ]:
./bin/basta sequence INPUT_FILE OUTPUT_FILE MAPPING_FILE_TYPE

In [ ]:
./scripts/filter_basta_fasta.py [options] FASTA_FILE FILTERED_OUTPUT_FILE NAME_OF_TAXON BASTA_FILE

# 

## genome quality

In [5]:
Path("{}/results/busco".format(proj_dir)).mkdir(parents=True, exist_ok=True)
os.chdir("{}/results/busco".format(proj_dir))

In [10]:
for sample in samples:
    assembly = "{}/results/polish_assembly/{}.assembly_rnd3.fasta".format(proj_dir, sample)
    !conda run -n busco --cwd . --live-stream busco -i {assembly} --auto-lineage-euk -c 48 -o {sample} -m genome 

2023-02-07 16:58:55 INFO:	***** Start a BUSCO v5.4.3 analysis, current time: 02/07/2023 16:58:55 *****
2023-02-07 16:58:55 INFO:	Configuring BUSCO with local environment
2023-02-07 16:58:55 INFO:	Mode is genome
2023-02-07 16:58:55 INFO:	Downloading information on latest versions of BUSCO data...
2023-02-07 16:58:57 INFO:	Input file is /master/nplatt/sm_nanopore/results/polish_assembly/smor.assembly_rnd3.fasta
2023-02-07 16:58:57 INFO:	No lineage specified. Running lineage auto selector.

2023-02-07 16:58:57 INFO:	***** Starting Auto Select Lineage *****
	This process runs BUSCO on the generic lineage datasets for the domains archaea, bacteria and eukaryota. Once the optimal domain is selected, BUSCO automatically attempts to find the most appropriate BUSCO dataset to use based on phylogenetic placement.
	--auto-lineage-euk and --auto-lineage-prok are also available if you know your input assembly is, or is not, an eukaryote. See the user guide for more information.
	A reminder: Busco e

## genome quality

In [ ]:
for sample in samples:
    assembly = "{}/polish_assembly/{}.assembly_rnd3.fasta".format(proj_dir, sample)
    ./JupiterPlot/jupiter t=48 name={sample} ng=85 ref={ref_fas} fa={assembly}

## Grave Yard

In [ ]:
conda activate sm_ont-process_reads

cd /master/nplatt/sm_nanopore

cd sm_nanopore/results/nanofilt
3
 ~/anaconda3/envs/sm_ont-process_reads/bin/flye --nano-cor ../read_correction_canu/${SAMPLE}/${SAMPLE}.correctedReads.fasta.gz -g 385m -t 96  -i 3 -o ${SAMPLE} >${SAMPLE}.canu.log 2>&1 &

conda activate repeatmasker
RepeatMasker \
    -pa 12 \
    -lib ../repeatmodeler/Smansoni_v7-families.fa \
    -gff \
    -s \
    -dir $(pwd) \
    -small \
    -x \
    $REF



SmOR_Female_6_02_22_vSM_V9_sorted.vcf
schisto_pairs_04132022_vSM_V9_sorted.vcf
SmEG_Female_5_25_22_vSM_V9_sorted.vcf

bcftools filter \
    -sLOWFREQ_SMALL -i' (SVLEN<-500 || SVLEN>500) && AF>0.75' \
    schisto_pairs_04132022_vSM_V9_sorted.vcf \
    | vcftools \
        --remove-filtered-all \
        --vcf - \
        --recode \
        --recode-INFO-all \
        --stdout \
        >smle-pzq-es_filtered.vcf

bedtools intersect \
    -c \
    -b smle-pzq-es_filtered.vcf \
    -a ../lift/v9.bed \
    >smle-pzq-es_intersect.bed


bcftools filter \
    -sLOWFREQ_SMALL -i' (SVLEN<-200 || SVLEN>200)' \
    schisto_pairs_04132022_vSM_V9_sorted.vcf \
    | vcftools \
        --remove-filtered-all \
        --vcf - \
        --recode \
        --recode-INFO-all \
        --stdout

#############################################################################################

#genrate reads
art_illumina \
    --len 150 \
    --fcov 100 \
    --errfree \
    -na \
    --in SM_V9.fa \
    --out sm_v9_illumina \
    >sm_v9_illumina.log 2>&1 &

nanosim-h \
    --perfect \
    -o sm_v9_ont \
    -n 5000000 \
    SM_V9.fa \
    >sm_v9_ont.log 2>&1 &

#convert to fasta
sed -n '1~4s/^@/>/p;2~4p' sm_v9_illumina.fq > sm_v9_illumina.fa

#make sure amount of data is similar
echo sm_v9_ont.fa $(cat sm_v9_ont.fa | paste  - - | cut -f2 | wc -c) >>counts &
echo sm_v9_illumina.fa $(cat sm_v9_illumina.fa | paste  - - | cut -f2 | wc -c) >>counts &




#map reads (perfectly and unambiguosly) back to the reference
for SAMPLE in sm_v9_illumina sm_v9_ont; do
    bbmap.sh \
        ref=SM_V9.fa \
        perfectmode=t \
        threads=48 \
        ambiguous=toss \
        in=${SAMPLE}.fa \
        overwrite=t \
        out=${SAMPLE}.sam \
        mappedonly=t \
        >${SAMPLE}.bbmap.log 2>&1 &
done


#calculate coverage
for SAMPLE in sm_v9_illumina sm_v9_ont; do

    echo $SAMPLE
    echo "...sort"
    samtools view -b -F 4 ${SAMPLE}.sam | samtools sort >${SAMPLE}.bam
    echo "...idx"
    samtools index ${SAMPLE}.bam
    echo "...cov"
    mosdepth \
        --threads 4 \
        ${SAMPLE} \
        ${SAMPLE}.bam
done

#how much is inaccessible per read type
for SAMPLE in sm_v9_illumina sm_v9_ont; do

    MB=$(zcat ${SAMPLE}.per-base.bed.gz | awk '{if ($4<=2) SUM+=$3-$2} END {print SUM/1000000}')
    echo -e $SAMPLE"\t"$MB

done

art_illumina \
--paired \
--len 150 \
--fcov 10 \
--mflen 300 \
--sdev 80 \
-na \
--in SM_V9.fa \
--out sm_v9_wes


bbmap.sh \
    ref=SM_V9.fa \
    threads=48 \
    ambiguous=toss \
    in=sm_v9_wes1.fq \
    in2=sm_v9_wes2.fq \
    samestrandpairs=t \
    killbadpairs=t \
    overwrite=t \
    out=sm_v9_wes.sam \
    mappedonly=t \
    ignorebadquality=t

samtools view -b -F 4 sm_v9_wes.sam | samtools sort >sm_v9_wes.bam
samtools index sm_v9_wes.bam

mosdepth \
    --threads 4 \
    sm_v9_wes \
    sm_v9_wes.bam



zcat sm_v9_wes.per-base.bed.gz | awk '{if ($4>=12) SUM+=$3-$2} END {print SUM/1000000}'
zcat sm_v9_wes.per-base.bed.gz | awk '{if ($4<=2) SUM+=$3-$2} END {print SUM/1000000}'








sed -i 's/SM_V9_1/HE601624.3/' sm_v9.gff3
sed -i 's/SM_V9_2/HE601625.3/' sm_v9.gff3
sed -i 's/SM_V9_3/HE601626.3/' sm_v9.gff3
sed -i 's/SM_V9_4/HE601627.3/' sm_v9.gff3
sed -i 's/SM_V9_5/HE601628.3/' sm_v9.gff3
sed -i 's/SM_V9_6/HE601629.3/' sm_v9.gff3



bedtools intersect \
    -wb \
    -a smle-pzq-es_filtered.vcf \
    -b sm_v9.gff3





bcftools filter \
    -sLOWFREQ_SMALL -i' (SVLEN<-500 || SVLEN>500) && AF>0.75' \
    schisto_pairs_04132022_vSM_V9_sorted.vcf \
    | vcftools \
        --remove-filtered-all \
        --vcf - \
        --recode \
        --recode-INFO-all \
        --stdout \
        >smle-pzq-es_filtered.vcf

bedtools intersect \
    -c \
    -b smle-pzq-es_filtered.vcf \
    -a ../lift/v9.bed








cp ../cutesv/sm_v9.gff3 .
awk '{if ($3=="exon") print $0}' sm_v9.gff3 >sm_v9_exons.gff3
cut -f1 -d";" sm_v9_exons.gff3 | cut -f1,4,5,9 | sed 's/ID=exon://' >sm_v9_exons.bed
bedtools intersect -a sm_v9_exons.bed  -b qtls_v9.bed >qtl_exons.bed


bedtools intersect -wb -a qtls_v9.bed -b smeg_filtered.vcf  >smeg_qtl_snvs.vcf
bedtools intersect -wa -b smeg_qtl_snvs.vcf -a qtl_exons.bed   | sort -u -k4,4 > smeg_snvs_in_qtl_exons.bed
bedtools intersect -c -b smeg_snvs_in_qtl_exons.bed -a qtls_v9.bed
bedtools intersect -wb -b smeg_snvs_in_qtl_exons.bed -a qtls_v9.bed | cut -f4,8 | cut -f1 -d"." | sort -u -k2,2 | cut -f1 | sort | uniq -c

bedtools intersect -wb -a qtls_v9.bed -b smor_filtered.vcf  >smor_qtl_snvs.vcf
bedtools intersect -wa -b smor_qtl_snvs.vcf -a qtl_exons.bed   | sort -u -k4,4 > smor_snvs_in_qtl_exons.bed
bedtools intersect -c -b smor_snvs_in_qtl_exons.bed -a qtls_v9.bed
bedtools intersect -wb -b smor_snvs_in_qtl_exons.bed -a qtls_v9.bed | cut -f4,8 | cut -f1 -d"." | sort -u -k2,2 | cut -f1 | sort | uniq -c


bedtools intersect -wb -a qtls_v9.bed -b smle-pzq-es_filtered.vcf  >smle-pzq-es_qtl_snvs.vcf
bedtools intersect -wa -b smle-pzq-es_qtl_snvs.vcf -a qtl_exons.bed   | sort -u -k4,4 > smle-pzq-es_snvs_in_qtl_exons.bed
bedtools intersect -c -b smle-pzq-es_snvs_in_qtl_exons.bed -a qtls_v9.bed
bedtools intersect -wb -b smle-pzq-es_snvs_in_qtl_exons.bed -a qtls_v9.bed | cut -f4,8 | cut -f1 -d"." | sort -u -k2,2 | cut -f1 | sort | uniq -c









nanosim-h --perfect -o sm_v9_ont_reads -n 500000 SM_V9.fa


    minimap2 \
        -a \
        -o sm_v9_ont.sam \
        -t 48 \
        -x map-ont \
        SM_V9.fa \
        sm_v9_ont_reads.fa



samtools view -b -F 4 sm_v9_ont.sam | samtools sort >sm_v9_ont.bam

samtools index sm_v9_ont.bam

mosdepth \
    --threads 4 \
    sm_v9_ont \
    sm_v9_ont.bam

zcat sm_v9_ont.per-base.bed.gz | awk '{if ($4>=12) SUM+=$3-$2} END {print SUM/1000000}'
zcat sm_v9_ont.per-base.bed.gz | awk '{if ($4<=2) SUM+=$3-$2} END {print SUM/1000000}'


bbmap.sh \
    ref=SM_V9.fa \
    threads=48 \
    ambiguous=toss \
    in=sm_v9_ont_reads.fa \
    overwrite=t \
    out=sm_v9_ont_bbmap.sam \
    mappedonly=t \
    ignorebadquality=t

bbmap.sh \
    ref=SM_V9.fa \
    perfectmode=t \
    threads=48 \
    ambiguous=toss \
    in=sm_v9_ont_reads.fa \
    overwrite=t \
    out=sm_v9_ont_bbmap.sam \
    mappedonly=t


samtools view -b -F 4 sm_v9_ont_bbmap.sam | samtools sort >sm_v9_ont_bbmap.bam
samtools index sm_v9_ont_bbmap.bam

mosdepth \
    --threads 4 \
    sm_v9_ont_bbmap \
    sm_v9_ont_bbmap.bam

zcat sm_v9_ont_bbmap.per-base.bed.gz | awk '{if ($4>=12) SUM+=$3-$2} END {print SUM/1000000}'
zcat sm_v9_ont_bbmap.per-base.bed.gz | awk '{if ($4<=2) SUM+=$3-$2} END {print SUM/1000000}'

RepeatMasker -qq -pa 48 -nolow -norna -dir . -gff -lib sm_v7_tes.fas SM_V9.fa

SmEG_Female_5_25_22
schisto_pairs_04132022  ../cutesv/SmOR_Female_6_02_22_vSM_V9_sorted.vcf

for VCF in $(ls ../cutesv/*_vSM_V9_sorted.vcf); do 

bcftools filter \
    -sLOWFREQ_SMALL -i' (SVLEN<-500 || SVLEN>500)' \
    ../cutesv/schisto_pairs_04132022_vSM_V9_sorted.vcf \
    | vcftools \
        --remove-filtered-all \
        --vcf - \
        --recode \
        --recode-INFO-all \
        --stdout \
        >smle-pze-es_bigsvs.vcf

bcftools filter \
    -sLOWFREQ_SMALL -i' (SVLEN<-500 || SVLEN>500)' \
    ../cutesv/SmEG_Female_5_25_22_vSM_V9_sorted.vcf \
    | vcftools \
        --remove-filtered-all \
        --vcf - \
        --recode \
        --recode-INFO-all \
        --stdout \
        >smeg_bigsvs.vcf

bcftools filter \
    -sLOWFREQ_SMALL -i' (SVLEN<-500 || SVLEN>500)' \
    ../cutesv/SmOR_Female_6_02_22_vSM_V9_sorted.vcf \
    | vcftools \
        --remove-filtered-all \
        --vcf - \
        --recode \
        --recode-INFO-all \
        --stdout \
        >smor_bigsvs.vcf

   396,000,000
39,777,694,771
